### 필요한 라이브러리 import

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup as bs

### 결과물 형태

In [2]:
result=pd.read_csv('data/시청률.csv',usecols=('회차','방영일','닐슨','TNMS'))
result.head()

,회차,방영일,닐슨,TNMS
0,433회,1.06,7.3%,8.2%
1,434회,1.13,6.4%,7.2%
2,435회,1.20,7.3%,6.9%
3,436회,1.27,6.0%,7.6%
4,437회,2.03,6.2%,7.1%


In [3]:
url = 'https://namu.wiki/w/%EB%9F%B0%EB%8B%9D%EB%A7%A8/2019%EB%85%84'
res = requests.get(url)
soup = bs(res.content,'html.parser')

콘텐츠 구조가 아래 두 태그가 반복되는 형태
- h2 class='wiki-heading' 
- div class='wiki-heading-content' 

-> 따라서 3. 시청률 추이 콘텐츠는 3번째 반복될 때 나오고 인덱스로 2

In [4]:
data=soup.select('.wiki-heading-content')[2]

해당 div 안에 시청률 데이터가 table 형태로 제공되고 tr > td 안에 한줄로 데이터가 들어있음

In [5]:
table=data.find('table',{'class':'wiki-table'}).find_all('td')

In [6]:
print(len(table))
print(table[0].text)
print(table[1].text)
print(table[2].text)
print(table[3].text)

208
회차
방영일
닐슨
TNMS


In [7]:
table

[<td style="text-align:center;"><div class="wiki-paragraph"><span class="wiki-color" style="color:#FFFFFF"><strong>회차</strong></span></div></td>,
 <td style="text-align:center;"><div class="wiki-paragraph"><span class="wiki-color" style="color:#FFFFFF"><strong>방영일</strong></span></div></td>,
 <td style="text-align:center;"><div class="wiki-paragraph"><span class="wiki-color" style="color:#FFFFFF"><strong>닐슨</strong></span></div></td>,
 <td style="text-align:center;"><div class="wiki-paragraph"><span class="wiki-color" style="color:#FFFFFF"><strong>TNMS</strong></span></div></td>,
 <td data-dark-style="background-color:#BD9700;" style="background-color:#FFDA41; text-align:center;"><div class="wiki-paragraph">433회</div></td>,
 <td style="text-align:center;"><div class="wiki-paragraph">01.06</div></td>,
 <td style="text-align:center;"><div class="wiki-paragraph">7.3%</div></td>,
 <td style="text-align:left;"><div class="wiki-paragraph">8.2%</div></td>,
 <td data-dark-style="background-col

In [21]:
회차 = []
방영일 = []
닐슨 = []
TNMS = []

In [22]:
def make_list(n,list_name):    
    for t in range(n,len(table),4):
        list_name.append(table[t].text)

In [23]:
make_list(4,회차)
make_list(5,방영일)
make_list(6,닐슨)
make_list(7,TNMS)

In [24]:
df=pd.DataFrame({'회차':회차,
             '방영일':방영일,
             '닐슨':닐슨,
             'TNMS':TNMS})

### Plotly 시각화

<a href='https://plotly.com/python/line-charts/'> plotly 참고 </a>

In [36]:
import plotly.express as px
import plotly.graph_objects as go

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [26]:
# 방영일 str -> datetime 으로 변경
df['방영일']=df['방영일'].apply(lambda x:'2020-'+x.split('.')[0]+'-'+x.split('.')[1])
df['방영일']

0     2020-01-06
1     2020-01-13
2     2020-01-20
3     2020-01-27
4     2020-02-03
5     2020-02-10
6     2020-02-17
7     2020-02-24
8     2020-03-03
9     2020-03-10
10    2020-03-17
11    2020-03-24
12    2020-03-31
13    2020-04-07
14    2020-04-14
15    2020-04-21
16    2020-04-28
17    2020-05-05
18    2020-05-12
19    2020-05-19
20    2020-05-26
21    2020-06-02
22    2020-06-09
23    2020-06-16
24    2020-06-23
25    2020-07-07
26    2020-07-14
27    2020-07-21
28    2020-07-28
29    2020-08-04
30    2020-08-11
31    2020-08-18
32    2020-08-25
33    2020-09-01
34    2020-09-08
35    2020-09-15
36    2020-09-22
37    2020-09-29
38    2020-10-06
39    2020-10-13
40    2020-10-20
41    2020-10-27
42    2020-11-03
43    2020-11-10
44    2020-11-17
45    2020-11-24
46    2020-12-01
47    2020-12-08
48    2020-12-15
49    2020-12-22
50    2020-12-29
Name: 방영일, dtype: object

In [29]:
df['방영일']=pd.to_datetime(df['방영일'],format = '%Y-%m-%d')

In [133]:
#fig=px.line(df, x='방영일', y=['닐슨','TNMS'] ,title = '런닝맨 시청률(닐슨)')
#fig.show()

* 12월 8일 TNMS 시청율에 특이값-> 8.4%를 8,4%로 오타 낸 것으로 보임

In [119]:
# 오타 수정
#df.loc[df['방영일'] =='12.08','TNMS'] = '8.5%'

In [48]:
fig=px.line(df, x='방영일', y=['닐슨','TNMS'] ,title = '런닝맨 시청률(닐슨&TNMS)',line_shape='linear', 
            labels={
                     "방영일": "방영일",
                     "value": "시청율",
                     "variable": "조사기관"
                 })
fig.show()